# SQLite3 활용

2020.11.06
파이썬3내장되어있는 mysql SQLite  
http://www.sqliteexpert.com/download.html  
퍼스널-64bit다운  
test.db 폴더만들었떠니  생성! 우앙!!  
SQL탭에서 작성  (테이블생성)  
create table test (  
  id integer primary key autoincrement,   
  name text not null   
);  
하단에 익스큐트 누르니 생성!!  

정보를 넣어보자   
insert into test(name) values ('홍길동'), ('대조영');  
select * from test;  


VScode에서는 작성할때   
conn = sqlite3.connect('./test.db')  
cur = conn.cursor() 꼭하고  
변경한다음에는 conn.commit()  
에스큐라이트 update test set age=300 where id>=1; 작성하면 당연 반영되겠지  

### 접속

In [6]:
import sqlite3
conn = sqlite3.connect('./test.db') #현폴더에있는 테스트쩜디비를 열어 연결하라

# 작업실행 후 
# db access 하고
cur = conn.cursor()                #커서 객체등록? 받음?
cur.execute('select * from test;')  # 실행하고
#읽는방법은
rows = cur.fetchall()
for row in rows:
    print(row)

conn.close()

(1, '홍길동')
(2, '대조영')


In [10]:
conn = sqlite3.connect('./test.db')
with conn:
    cur = conn.cursor()
    cur.execute('select * from test;')
    rows = cur.fetchall()
    for row in rows:
        print(row)
        #클로즈는 제일제일 마지막에 하면 됨

(1, '홍길동')
(2, '대조영')


In [11]:
conn = sqlite3.connect('./test.db')
cur = conn.cursor()

In [12]:
#테이블 구조 변경
sql = 'alter table test add column age integer;'
cur.execute(sql)

In [13]:
conn.commit()   #변경사항 저장할때 .. SQLite에 가서 확인하면 age컬럼 추가되어있음

### 데이터 추가

In [14]:
cur = conn.cursor()
sql = "insert into test(name, age) values('장보고', 1000);"
cur.execute(sql)
conn.commit()

In [ ]:
# SQLite에서  select * from test; 확인하면 장보고 1000 들어가있음

In [15]:
#여러개 입력해보자
#파라메타는 튜플로 넘겨준다
writers = [('이우정', 40), ('김은숙', 45)]   
sql = "insert into test(name, age) values(?, ?);"  #파라메타는 ?,? 이걸로받고
for writer in writers:
    cur.execute(sql, writer)
conn.commit()

In [16]:
cur.execute(sql, ('김수현', 60)) #파라메타는 튜플로 전달한다는거~
conn.commit()

In [18]:
#리스트 안에 튜플로 3명 넣어보자 __ 전달할 내용이 튜플이면 됨.
writers = [('노희경', 30), ('송지나', 35), ('임성한', 40)]
cur.executemany(sql, writers)     #그냥 익스큐트아니고 익스큐트매니!!!
conn.commit()

## 데이터조회

In [27]:
# 단건 조회
cur = conn.cursor()
select_sql = 'select * from test;'
cur.execute(select_sql)  #실행만 하고, 데이터를 안가지고 온상태
row = cur.fetchone()    #이것이 데이터를 가져오는 방법!!!
print(row)

(1, '홍길동', 300)


In [28]:
# 여러건 조회 fetchmany() 
#  위에 sql실행한 상태. 커서가 두번째줄에 위치해있어서
rows = cur.fetchmany(3)  #3건 부르니까 #2.3.4줄에있는 대조영장보고이우정 불려옴
for row in rows: 
    print(row)

(2, '대조영', 300)
(3, '장보고', 1000)
(4, '이우정', 40)


In [29]:
rows = cur.fetchmany(2)  
for row in rows: 
    print(row)

(5, '김은숙', 45)
(6, '김수현', 60)


In [30]:
# 나머지 모두 조회 fetchall()
rows = cur.fetchall()
for row in rows: 
    print(row)

(7, '노희경', 30)
(8, '송지나', 35)
(9, '임성한', 40)


In [31]:
#모두 조회
cur = conn.cursor()
cur.execute(select_sql)  
rows = cur.fetchall()
for row in rows: 
    print(row)

(1, '홍길동', 300)
(2, '대조영', 300)
(3, '장보고', 1000)
(4, '이우정', 40)
(5, '김은숙', 45)
(6, '김수현', 60)
(7, '노희경', 30)
(8, '송지나', 35)
(9, '임성한', 40)


In [32]:
# 순회조회
cur = conn.cursor()
cur.execute(select_sql)  
for row in cur:  #cur로바꾸면 커서가 순회하게됨 
    print(row)

(1, '홍길동', 300)
(2, '대조영', 300)
(3, '장보고', 1000)
(4, '이우정', 40)
(5, '김은숙', 45)
(6, '김수현', 60)
(7, '노희경', 30)
(8, '송지나', 35)
(9, '임성한', 40)


In [34]:
# 이제 조건을 줄게....내가필요한부분만가져와~  ppt 5.데이터조작언어 데이터조회부분
# 필요한 컬럼culumn만 조회
cur = conn.cursor()
cur.execute("select name, age from test order by name;")   #이름 가나다순으로
rows = cur.fetchall()
for row in rows: 
    print(row)

('김수현', 60)
('김은숙', 45)
('노희경', 30)
('대조영', 300)
('송지나', 35)
('이우정', 40)
('임성한', 40)
('장보고', 1000)
('홍길동', 300)


In [35]:
# 원하는 순서 및 갯수 조건2) 연장자 3명만 가져와~
cur = conn.cursor()
cur.execute("select * from test order by age desc limit 3;")   #연장자 TOP3 홍길동대조영인데
rows = cur.fetchall()
for row in rows: 
    print(row)

(3, '장보고', 1000)
(1, '홍길동', 300)
(2, '대조영', 300)


In [36]:
# 원하는 순서 및 갯수 조건2) 연장자 3명만 가져와~
cur = conn.cursor()  #이거 안넣어도 에러가 나지는 않는구나~~
cur.execute("select * from test order by age desc, name limit 3;")   #연장자 TOP3 이름순으로 300나이를 정렬.  ,name추가
rows = cur.fetchall()
for row in rows: 
    print(row)

(3, '장보고', 1000)
(2, '대조영', 300)
(1, '홍길동', 300)


In [53]:
# db에 몇건있는지 함수를 써서알아보자
# 함수1
cur.execute('select count(*) from test;')
count = cur.fetchone()  # 튜플언패킹 count,  해도되고 그냥 count해도되고
print(count, count[0])            # 튜플입니다. 그래서 count[0]

(9,) 9


In [54]:
# 함수2
cur.execute('select max(age), min(age), sum(age), avg(age) from test;')
row = cur.fetchone()
print(row)

(1000, 30, 1850, 205.55555555555554)


In [55]:
max_age = row[0]
min_age = row[1]
sum_age = row[2]
avg_age = row[3]

In [56]:
# 이렇게 해도 된다~~ 
cur.execute('select max(age), min(age), sum(age), avg(age) from test;')
max_age, min_age, sum_age, avg_age = cur.fetchone()
print(max_age, min_age, sum_age, avg_age)  

1000 30 1850 205.55555555555554


## 데이터 검색

In [58]:
# 주의주의!! 검색할때 튜플로 줄것!!!!!!!!!
cur = conn.cursor()
cur.execute('select * from test where age < ?', (50,)) #50세 미만을 찾고싶을때, 튜플로 (50,) 해야 데이터 전달됨
rows = cur.fetchall()
for row in rows:
    print(row)

(4, '이우정', 40)
(5, '김은숙', 45)
(7, '노희경', 30)
(8, '송지나', 35)
(9, '임성한', 40)


## 데이터 변경

In [59]:
cur.execute('update test set age=1300 where id=2;')
conn.commit()
#대조영 1000살이 대조영1300살 되셨다

## 데이터 삭제

In [60]:
cur.execute('delete from test where id=6;')
conn.commit()
# 6.김수현님 bye.............. 

## 2020.11.06 과제 PPT.11참고